# Self-Driving Car Engineer Nanodegree

## Project: Finding Lane Lines on the Road

### Import Packages

In [1]:
import numpy as np
import cv2
from collections import deque

### Function Define

In [2]:
def nothing(x):
    pass
#This function is used for the fifth parameter of function cv2.createTrackbar()

### Read in an video and set parameters for video reading and writing

In [3]:
cap = cv2.VideoCapture("solidWhiteRight.mp4")
fps = cap.get(5)
size = (int(cap.get(3)), int(cap.get(4)))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out_comb = cv2.VideoWriter("output_solidWhiteRight.avi", fourcc, fps, size, isColor=True)

### Initial setting of parameters

In [4]:
cv2.namedWindow('parameters')
cv2.createTrackbar('kernel_size', 'parameters', 1, 50, nothing)
cv2.createTrackbar('low_threshold', 'parameters', 50, 255, nothing)
cv2.createTrackbar('high_threshold', 'parameters', 150, 255, nothing)
cv2.createTrackbar('rho', 'parameters', 2, 50, nothing)
cv2.createTrackbar('theta', 'parameters', 1, 180, nothing)
cv2.createTrackbar('point_threshold', 'parameters', 10, 500, nothing)
cv2.createTrackbar('min_line_length', 'parameters', 30, 500, nothing)
cv2.createTrackbar('max_line_gap', 'parameters', 20, 500, nothing)
cv2.createTrackbar('left_limit', 'parameters', 480, 480, nothing)
cv2.createTrackbar('right_limit', 'parameters', 480, 959, nothing)
cv2.createTrackbar('high_limit', 'parameters', 320, 539, nothing)
cv2.createTrackbar('bottom_limit', 'parameters', 500, 539, nothing)
cv2.createTrackbar('left_bottom', 'parameters', 100, 480, nothing)
cv2.createTrackbar('right_bottom', 'parameters', 859, 959, nothing)
cv2.createTrackbar('w_H_L', 'parameters', 0, 179, nothing)
cv2.createTrackbar('w_S_L', 'parameters', 0, 255, nothing)
cv2.createTrackbar('w_V_L', 'parameters', 200, 255, nothing)
cv2.createTrackbar('w_H_H', 'parameters', 100, 179, nothing)
cv2.createTrackbar('w_S_H', 'parameters', 30, 255, nothing)
cv2.createTrackbar('w_V_H', 'parameters', 255, 255, nothing)
cv2.createTrackbar('Y_H_L', 'parameters', 20, 179, nothing)
cv2.createTrackbar('Y_S_L', 'parameters', 150, 255, nothing)
cv2.createTrackbar('Y_V_L', 'parameters', 100, 255, nothing)
cv2.createTrackbar('Y_H_H', 'parameters', 30, 179, nothing)
cv2.createTrackbar('Y_S_H', 'parameters', 255, 255, nothing)
cv2.createTrackbar('Y_V_H', 'parameters', 255, 255, nothing)
#For the purpose of adjusting parameters in real-time, I create trackbars for most of the parameters.
#The initial values of each parameters are the best values as I know according to many tries.

fps_continue = 5
slope_list_left = deque([], maxlen=fps_continue)
slope_list_right = deque([], maxlen=fps_continue)
x_list_left = deque([], maxlen=fps_continue)
y_list_left = deque([], maxlen=fps_continue)
x_list_right = deque([], maxlen=fps_continue)
y_list_right = deque([], maxlen=fps_continue)
#For purpose to restrain shaking of annotated lan lines, I used queues of slopes and middle points of continuous 5 fps
#and so that the slopes and middle points for current fps is acurally the mean value of 5 fps.

### The main while process 

In [5]:
while(cap.isOpened()):
    ret, image = cap.read()
    if ret:
        kernel_size = 3 + 2 * cv2.getTrackbarPos('kernel_size', 'parameters')
        low_threshold = cv2.getTrackbarPos('low_threshold', 'parameters')
        high_threshold = cv2.getTrackbarPos('high_threshold', 'parameters')
        rho = cv2.getTrackbarPos('rho', 'parameters')
        theta = cv2.getTrackbarPos('theta', 'parameters') * np.pi / 180
        threshold = cv2.getTrackbarPos('threshold', 'parameters')
        min_line_length = cv2.getTrackbarPos('min_line_length', 'parameters')
        max_line_gap = cv2.getTrackbarPos('max_line_gap', 'parameters')
        left_limit = cv2.getTrackbarPos('left_limit', 'parameters')
        right_limit = cv2.getTrackbarPos('right_limit', 'parameters')
        high_limit = cv2.getTrackbarPos('high_limit', 'parameters')
        bottom_limit = cv2.getTrackbarPos('bottom_limit', 'parameters')
        left_bottom = cv2.getTrackbarPos('left_bottom', 'parameters')
        right_bottom = cv2.getTrackbarPos('right_bottom', 'parameters')
        Y_H_L = cv2.getTrackbarPos('Y_H_L', 'parameters')
        Y_S_L = cv2.getTrackbarPos('Y_S_L', 'parameters')
        Y_V_L = cv2.getTrackbarPos('Y_V_L', 'parameters')
        Y_H_H = cv2.getTrackbarPos('Y_H_H', 'parameters')
        Y_S_H = cv2.getTrackbarPos('Y_S_H', 'parameters')
        Y_V_H = cv2.getTrackbarPos('Y_V_H', 'parameters')
        W_H_L = cv2.getTrackbarPos('w_H_L', 'parameters')
        W_S_L = cv2.getTrackbarPos('w_S_L', 'parameters')
        W_V_L = cv2.getTrackbarPos('w_V_L', 'parameters')
        W_H_H = cv2.getTrackbarPos('w_H_H', 'parameters')
        W_S_H = cv2.getTrackbarPos('w_S_H', 'parameters')
        W_V_H = cv2.getTrackbarPos('w_V_H', 'parameters')
#Firstly, read in the values from track bars for each parameters.

        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lower_yellow = np.array([Y_H_L, Y_S_L, Y_V_L])
        upper_yellow = np.array([Y_H_H, Y_S_H, Y_V_H])
        mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
        """
        lower_white = np.array([W_H_L, W_S_L, W_V_L])
        upper_white = np.array([W_H_H, W_S_H, W_V_H])
        mask_white = cv2.inRange(hsv, lower_white, upper_white)
        """
        ret2, mask_white2 = cv2.threshold(gray, 200, 0, cv2.THRESH_TOZERO)
        mask_color = cv2.bitwise_or(mask_white2, mask_yellow)
        color_select = cv2.bitwise_and(image, image, mask=mask_color)
        cv2.imshow('image1_color_select', color_select)
#The first process I choosed is the color selection. I made a yellow mask and a white mask respectively, and then
#combine them by a bitwise_or operation.
#For the white mask, two way can achieve it. One is the way same with yellow mask, by using the HSV color space. 
#The other is by the THRESH_TOZERO threshold operation. In my opinion, especially for the colors black and white, 
#threshold operation usually easy to be used and have a better effection.

        gray = cv2.cvtColor(color_select, cv2.COLOR_BGR2GRAY)
        blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
        edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
        cv2.imshow('image2_edges_detect', edges)
        mask_region = np.zeros_like(edges)
        ignore_mask_color = 255
        imshape = image.shape
        vertices = np.array([[(left_bottom, bottom_limit), (left_limit, high_limit), (right_limit, high_limit), (right_bottom, bottom_limit)]], dtype=np.int32)
        cv2.fillPoly(mask_region, vertices, ignore_mask_color)
        masked_edges = cv2.bitwise_and(edges, mask_region)
        cv2.imshow('image3_masked_edges', masked_edges)
#The second process is the canny edges detection and region selection based on the image processed by color selection.
#Owning to the color selection, canny edges detection output an result without many noise. And by the region selection,
#almost all of the remaining noise can be removed.

        slope_left = 0#The mean value of all left line slopes in one fps 
        slope_right = 0#The mean value of al right line slopes in one fps
        n_left = 0#In one fps, how many left lines have been detected
        n_right = 0#In one fps, how many right lines have been detected
        x_left = 0#The middle point of all left lines in one fps
        y_left = 0#The middle point of all left lines in one fps
        x_right = 0#The middle point of all right lines in one fps
        y_right = 0#The middle point of all right lines in one fps
        line_image = np.copy(image) * 0
        lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
        for line in lines:
            for x1, y1, x2, y2 in line:
                if (x1 == x2) | (abs(y1 - y2) < 10):#To prevent 0 divide and get rid of horizontal lines
                    continue
                slope = (y2 - y1) / (x2 - x1)
                if slope > 0:
                    n_left += 1
                    if len(slope_list_left):
                        if abs(slope - sum(slope_list_left) / len(slope_list_left)) > 1:
                            slope = sum(slope_list_left) / len(slope_list_left)
                    slope_left += slope
                    x_left += (x1 + x2) / 2
                    y_left += (y1 + y2) / 2
                else:
                    n_right += 1
                    if len(slope_list_right):
                        if abs(slope - sum(slope_list_right) / len(slope_list_right)) > 1:
                            slope = sum(slope_list_right) / len(slope_list_right)
                    slope_right += slope
                    x_right += (x1 + x2) / 2
                    y_right += (y1 + y2) / 2

        if n_left & n_right:#To prevent 0 divide
            slope_left /= n_left
            x_left /= n_left
            y_left /= n_left
            slope_right /= n_right
            x_right /= n_right
            y_right /= n_right
            #Calculate the mean values in one fps 
            slope_list_left.append(slope_left)
            x_list_left.append(x_left)
            y_list_left.append(y_left)
            slope_list_right.append(slope_right)
            x_list_right.append(x_right)
            y_list_right.append(y_right)
            #Operating and maintaining queues
        if (len(slope_list_right) > 0) & (len(slope_list_left) > 0):#To prevent 0 divide
            slope_left = sum(slope_list_left) / len(slope_list_left)
            slope_right = sum(slope_list_right) / len(slope_list_right)
            x_left = sum(x_list_left) / len(x_list_left)
            y_left = sum(y_list_left) / len(y_list_left)
            x_right = sum(x_list_right) / len(x_list_right)
            y_right = sum(y_list_right) / len(y_list_right)
            #Calculate the mean values of continues 5 fps
            b_left = y_left - slope_left * x_left
            b_right = y_right - slope_right * x_right
            x_cross = int((b_right - b_left) / (slope_left - slope_right))
            y_cross = int(slope_left * x_cross + b_left)
            #Calculate the cross point of left and right lan lines
            delt_x = 30 * np.cos(np.arctan(slope_right))
            delt_y = 30 * np.sin(np.arctan(slope_right))
            x1 = int(x_cross - delt_x)
            x2 = int(x_cross - 13 * delt_x)
            y1 = int(y_cross - delt_y)
            y2 = int(y_cross - 13 * delt_y)
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 10)
            #Draw left lan line based on cross point
            delt_x = 30 * np.cos(np.arctan(slope_left))
            delt_y = 30 * np.sin(np.arctan(slope_left))
            x1 = int(x_cross + delt_x)
            x2 = int(x_cross + 13 * delt_x)
            y1 = int(y_cross + delt_y)
            y2 = int(y_cross + 13 * delt_y)
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 10)
            #Draw right lan lines based on cross point

        comb = cv2.addWeighted(image, 0.8, line_image, 1, 0)
        cv2.imshow('image4', comb)
        out_comb.write(comb)
#Finally, the Houghlines process is able to output lane lines only. And I combine it into the initial images for an 
#final output.

        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    else:
        break

### Release source used before

In [6]:
cap.release()
out_comb.release()
cv2.destroyAllWindows()